<a href="https://colab.research.google.com/github/tommy90112/AI-chat-robot/blob/main/%E8%82%A1%E7%A5%A8%E6%9F%A5%E8%A9%A2%E6%8E%A8%E8%96%A6RAG%E7%B3%BB%E7%B5%B1(%E5%8F%B0%E8%82%A1_%E5%8D%8A%E5%B0%8E%E9%AB%94_%E9%9B%BB%E5%AD%90%E9%9B%B6%E7%B5%84%E4%BB%B6_ETF_%E9%87%91%E8%9E%8D_%E7%87%9F%E5%BB%BA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
URL = "https://drive.google.com/uc?export=download&id=1bwKlo0lYkGFIk4eFMwXLu72YLagBsgOK"
!wget -O faiss_db.zip "$URL"

--2025-12-23 13:32:04--  https://drive.google.com/uc?export=download&id=1bwKlo0lYkGFIk4eFMwXLu72YLagBsgOK
Resolving drive.google.com (drive.google.com)... 64.233.181.138, 64.233.181.102, 64.233.181.100, ...
Connecting to drive.google.com (drive.google.com)|64.233.181.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1bwKlo0lYkGFIk4eFMwXLu72YLagBsgOK&export=download [following]
--2025-12-23 13:32:04--  https://drive.usercontent.google.com/download?id=1bwKlo0lYkGFIk4eFMwXLu72YLagBsgOK&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.183.132, 2607:f8b0:4001:c64::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23643 (23K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]  23.09K  --.-KB/s   

In [ ]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
replace faiss_db/index.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### 1. 安裝載入必要套件

In [ ]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

In [ ]:
!pip install -U langchain-openai
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. 載入 `faiss_db`

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

### 4. 設定 LLM

In [ ]:
import os
from google.colab import userdata

我們這邊使用 Groq 的 API服務

模型的部分使用的是 llama 3.3 70b 的版本

In [ ]:
api_key = userdata.get('Groq')

In [ ]:
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
model = "llama-3.3-70b-versatile"
base_url="https://api.groq.com/openai/v1"

In [ ]:
client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

### 5. 指令設計

這邊給定的系統指令，本身是股票推薦諮詢師，根據用戶提的問題，給予回答及推薦。

（如：股票編號2882的基本資料、台積電與聯電的本益比分別為多少、推薦我高EPS的股票、想找高股淨值股票、推薦我ETF高淨值股票....等等）

並且有限定只能用繁體中文來回答，以免使用如英文等其他語言回答


In [ ]:
system_prompt = "你是股票推薦諮詢師，請根據資料來回應用戶的問題。只能使用台灣習慣的繁體中文回應。"

prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

使用者的問題是：{question}

請根據資料內容回覆，若資料不足請告訴用戶可以上網查詢，並告知深感抱歉後續會再添加資料上去。
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [ ]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 Web App

在展示對話匡textbox的部分，我們有給予用戶一些範例作爲參考

在提問時可以以這些作為例子下去更改提問

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 📈📉 股票推薦諮詢師")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入您想諮詢的問題...（如：台積電的本益比是多少？富邦的EPS是多少？ETF高淨值股推薦幾隻給我？股票編號5534基本資料？等等....）")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)